In [1]:
import requests
import json
import pandas as pd

def watson_request(text):
    base_url = "https://gateway.watsonplatform.net/tone-analyzer-experimental/api/v1/tone"

    r = requests.post(base_url, data = text, headers = {"Content-Type" : "text/plain"}, 
                      auth=("84a3bf89-c5f3-45b6-9ee2-5c6ee1d66fa5", "t95US4RGaQnX"))

    response = json.loads(r.text)
    return response

def create_row(response, text_id):
    row = {}
    row["Filename"] = text_id
    for tone in response['children']:
        tone_id = tone['id']
        for features in tone['children']:
            characteristic_id = tone_id + "_" + features['id']
            row[characteristic_id + "_" + "word_count"] = features['word_count']
            row[characteristic_id + "_" + "normalized_score"] = features['normalized_score']
            row[characteristic_id + "_" + "raw_score"] = features['raw_score']
    return row

watson_data_negative = []
import os
for subdir, dirs, files in os.walk('./op_spam_v1.4/negative_polarity/'):
    for file in files:
        text = open(str(subdir) + "/" + str(file)).read()
        response = watson_request(text)
        row = create_row(response, file)
        watson_data_negative.append(row)



watson_data_positive = []
import os
for subdir, dirs, files in os.walk('./op_spam_v1.4/positive_polarity/'):
    for file in files:
        text = open(str(subdir) + "/" + str(file)).read()
        response = watson_request(text)
        row = create_row(response, file)
        watson_data_positive.append(row)


watson_df_negative = pd.DataFrame(watson_data_negative)
watson_df_positive = pd.DataFrame(watson_data_positive)

In [3]:
print watson_df_negative
print watson_df_positive


                      Filename  emotion_tone_Anger_normalized_score  \
0               d_hilton_1.txt                             0.956234   
1              d_hilton_10.txt                             0.000000   
2              d_hilton_11.txt                             0.000000   
3              d_hilton_12.txt                             1.000000   
4              d_hilton_13.txt                             1.000000   
5              d_hilton_14.txt                             0.958082   
6              d_hilton_15.txt                             1.000000   
7              d_hilton_16.txt                             1.000000   
8              d_hilton_17.txt                             0.000000   
9              d_hilton_18.txt                             1.000000   
10             d_hilton_19.txt                             0.000000   
11              d_hilton_2.txt                             0.000000   
12             d_hilton_20.txt                             1.000000   
13    

In [4]:
import requests
import json
import pandas as pd
import os

def watson_request(text):
    base_url = "https://gateway.watsonplatform.net/tone-analyzer-experimental/api/v1/tone"

    r = requests.post(base_url, data = text, headers = {"Content-Type" : "text/plain"}, 
                      auth=("84a3bf89-c5f3-45b6-9ee2-5c6ee1d66fa5", "t95US4RGaQnX"))

    response = json.loads(r.text)
    return response

def create_row(response, text_id):
    row = {}
    row["Filename"] = text_id
    for tone in response['children']:
        tone_id = tone['id']
        for features in tone['children']:
            characteristic_id = tone_id + "_" + features['id']
            row[characteristic_id + "_" + "word_count"] = features['word_count']
            row[characteristic_id + "_" + "normalized_score"] = features['normalized_score']
            row[characteristic_id + "_" + "raw_score"] = features['raw_score']
    return row

def create_watson_db(polarity):
    watson_data = []
    filename = './op_spam_v1.4/'+str(polarity)+'_polarity/'
    for subdir, dirs, files in os.walk(filename):
        for file in files:
            text = open(str(subdir) + "/" + str(file)).read()
            response = watson_request(text)
            row = create_row(response, file)
            watson_data.append(row)
    watson_df = pd.DataFrame(watson_data)
    return watson_df

if __name__ =="__main__":
    watson_negative = create_watson_db('negative')
    watson_negative.to_csv('negative_watson.csv',sep=',',index=False)
    watson_positive = create_watson_db('positive')
    watson_positive.to_csv('positive_watson.csv',sep=',',index=False)
